In [6]:
import sys
sys.path.append('../')
import os
import impyute
from utils.read_file  import readNex,saveData,lableEncoder

from logger import logger
import time
import datetime
import tqdm
from utils.tools import saveJson
from baseline.SOTABaselineFixed import  imputeMethodFixed as Fixed
from baseline.SOTABaselineKNN import  imputeMethodKNN as KNN
from baseline.SOTABaselineMICE import  imputeMethodMICE as MICE
from baseline.SOTABaselineRandom import  imputeMethodRandom as Random
from baseline.SOTABaselineEM import  imputeMethodEM as EM
from baseline.SOTABaselineMedian import  imputeMethodMedain as Medain
from baseline.myMethod import imputeMethod as TAI
from baseline.SOTABaselineRF import imputeMethodMR as MR

from utils.handle_missingdata import gene_missingdata, gene_missingdata_taxa_bias, gene_missingdata_chara_bias, \
    gene_missingdata_block_bias


# 生成模拟无缺失数据集

In [7]:
originDataPath=r'C:\Users\pro\Desktop\实验二自编码器建树\古生物数据集测试\01起始数据集'
noMissingDataPath=r'C:\Users\pro\Desktop\实验二自编码器建树\古生物数据集测试\02随机插补无缺失数据集'
simDataPath=r'C:\Users\pro\Desktop\实验二自编码器建树\古生物数据集测试\03模拟缺失数据集'
#生成模拟数据集
# for file in os.listdir(originDataPath):
#     data,missRow,speciesName,begin,end=readNex(os.path.join(originDataPath,file))
#     noMissingData = impyute.imputation.cs.random(data)
#     saveData(noMissingDataPath,file,speciesName,noMissingData,begin,end)

In [ ]:
imputedDataPath = r'C:\Users\pro\Desktop\实验二自编码器建树\古生物数据集测试\04缺失插补结果'
for file in tqdm.tqdm(os.listdir(noMissingDataPath)):
    originData, missRow, speciesName, begin, end = readNex(os.path.join(noMissingDataPath, file))
    for missPattern in ['normal','block',  'taxa', 'chara']:
        result = {}
        for missRate in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
            if missPattern == 'normal':
                missData = gene_missingdata(rate=missRate, data=originData)
            elif missPattern == 'taxa':
                missData = gene_missingdata_taxa_bias(rate=missRate, data=originData)
            elif missPattern == 'chara':
                missData = gene_missingdata_chara_bias(rate=missRate, data=originData)
            elif missPattern == 'block':

                missData = gene_missingdata_block_bias(rate=missRate, data=originData)
            else:
                raise Exception("缺失模式错误，请在'normal','taxa','chara','block'中选择对应模式")
            saveData(simDataPath, "{}_{}_{}".format(missPattern,str(missRate),file), speciesName, missData, begin, end)
            missData = lableEncoder(missData)
            result, RandomImputedData = Random(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath, "Random"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     RandomImputedData, begin, end)
            result, MRImputedData = MR(result, originData, missData, missRate, missPattern)
            saveData(os.path.join(imputedDataPath, "MR"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     MRImputedData, begin, end)
            
            result, MedainImputedData = Medain(result, originData, missData, missRate, missPattern)
            saveData(os.path.join(imputedDataPath, "Medain"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     MedainImputedData, begin, end)
            result, EMImputedData = EM(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath, "EM"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     EMImputedData, begin, end)
            result, KNNImputedData = KNN(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath, "KNN"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     KNNImputedData, begin, end)
            
            result, FixedImputedData = Fixed(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath, "Fixed"), "{}_{}_{}".format(missPattern,str(missRate),file), speciesName,
                     KNNImputedData, begin, end)
            
            start = time.time()
            result, MICEImputedData = MICE(result, originData, missData, missRate, missPattern, 'disperse')
            firstImputedData = MICEImputedData
            firstImputedMethod='knn'
            for loss in ['MSELoss']:
                for autoMethod in ['Autoencoder']:
                    result, TAIimputedData = TAI(result=result, firstImputedMethod=firstImputedMethod,
                                                 firstImputedData=firstImputedData,
                                                 loss=loss, autoMethod=autoMethod,
                                                 originData=originData, missData=missData,
                                                 missRate=missRate, missPattern=missPattern,
                                                 dataType='disperse')
                    logger.info(
                        "改后{}-{}-{}训练耗时:{}".format(firstImputedMethod, loss, autoMethod, time.time() - start))
                    saveData(os.path.join(imputedDataPath, "ours"), "{}_{}".format(str(missRate),file), speciesName,
                             TAIimputedData, begin, end)
    saveJson(result,
             os.path.join(imputedDataPath, "{}_{}_{}_{}.json".format("allmethod", missPattern, file,
                                                                     datetime.datetime.now().strftime(
                                                                         '%Y%m%d-%H%M%S'))))

  0%|                                                                                                                                          | 0/4 [00:00<?, ?it/s]2021-01-29 22:20:37.131 | INFO     | utils.wapper:wapper:15 - imputeMethodRandom：耗时0.010001182556152344


In [ ]:
from utils.dataAnalysis import analysisMeanMain,analysisPlot
analysisMeanMain(savePath, savePath)